In [128]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np

In [129]:
df=pd.read_csv(r"C:\Users\vj753\Downloads\job_data.csv")

In [130]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vj753\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vj753\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vj753\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vj753\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [131]:
cols_to_drop = ['Job Id', 'latitude', 'longitude', 'Job Posting Date',
                'Preference', 'Contact Person', 'Contact', 'Job Portal']
df.drop(cols_to_drop, axis=1, inplace=True)

In [132]:
df = df.dropna()

In [133]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [134]:
df['']

,Experience,Qualifications,Salary Range,location,Country,Work Type,Company Size,Job Title,Role,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,5 to 12 Years,PhD,$57K-$125K,New Delhi,India,Full-Time,42159,Account Executive,Sales Account Executive,A Sales Account Executive is responsible for a...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Sales strategies and tactics Account managemen...,Identify and pursue sales opportunities with n...,GlaxoSmithKline,"{""Sector"":""Pharmaceuticals"",""Industry"":""Pharma..."
1,0 to 14 Years,BBA,$55K-$95K,New Delhi,India,Contract,102180,Customer Service Representative,Call Center Agent,A Call Center Agent handles incoming and outgo...,"{'Childcare Assistance, Paid Time Off (PTO), R...",Customer service Call center operations Proble...,Handle incoming phone calls and provide inform...,Textron,"{""Sector"":""Aerospace & Defense"",""Industry"":""Ae..."
2,4 to 13 Years,BBA,$55K-$108K,New Delhi,India,Full-Time,39814,Civil Engineer,Water Resources Engineer,A Water Resources Engineer manages and develop...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Water resources engineering Hydrology and hydr...,"Focus on water-related projects, such as flood...",Nucor,"{""Sector"":""Metals and Mining"",""Industry"":""Meta..."
3,1 to 8 Years,BA,$64K-$92K,New Delhi,India,Contract,48770,Veterinarian,Equine Veterinarian,An Equine Veterinarian specializes in the heal...,"{'Legal Assistance, Bonuses and Incentive Prog...",Equine veterinary medicine Horse anatomy and p...,Specialize in the care and treatment of horses...,Reliance Steel & Aluminum,"{""Sector"":""Metals"",""Industry"":""Metals"",""City"":..."
4,5 to 12 Years,BA,$56K-$124K,New Delhi,India,Intern,33699,Legal Counsel,Employment Lawyer,Employment Lawyers focus on employment law mat...,"{'Life and Disability Insurance, Stock Options...",Employment law and regulations Employment cont...,"Advise on employment law matters, including la...",Booz Allen Hamilton Holding,"{""Sector"":""Professional Services"",""Industry"":""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115935,0 to 12 Years,B.Tech,$64K-$114K,"Malabo (de jure),",Equatorial Guinea,Full-Time,18281,Mechanical Engineer,Mechanical Design Engineer,Mechanical Design Engineers create and develop...,"{'Employee Assistance Programs (EAP), Tuition ...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",The Hershey Company,"{""Sector"":""Food and Beverage/Confectionery"",""I..."
115936,2 to 14 Years,M.Tech,$62K-$130K,Warsaw,Poland,Intern,63621,IT Manager,IT Director,An IT Director oversees an organizations IT de...,"{'Health Insurance, Retirement Plans, Paid Tim...",Strategic IT planning Leadership and managemen...,Provide strategic leadership for IT department...,EQT,"{""Sector"":""Energy"",""Industry"":""Energy"",""City"":..."
115937,4 to 15 Years,BCA,$60K-$96K,Ashgabat,Turkmenistan,Part-Time,114287,Mechanical Engineer,Mechanical Design Engineer,Mechanical Design Engineers create and develop...,"{'Tuition Reimbursement, Stock Options or Equi...","Mechanical engineering CAD software (e.g., Sol...","Design mechanical systems, components, and pro...",KLA,"{""Sector"":""Technology"",""Industry"":""Semiconduct..."
115938,5 to 15 Years,BCA,$65K-$103K,Ouagadougou,Burkina Faso,Full-Time,45009,HR Coordinator,Training Coordinator,Training Coordinators design and implement emp...,"{'Casual Dress Code, Social and Recreational A...",Training program coordination Training materia...,"Coordinate employee training programs, track t...",Mahindra & Mahindra,"{""Sector"":""Automotive"",""Industry"":""Automotive""..."


In [135]:

categorical_cols = ['Qualifications', 'location', 'Country', 'Work Type',
                   'Job Title', 'Role', 'Company']
    
from sklearn.preprocessing import LabelEncoder

# Dictionary to store encoders
encoders = {}

# Fit encoders on the training set
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le  # Store the encoder for each column

In [136]:
# df['Qualifications']=label.fit_transform(df['Qualifications'])
# df['location']=label.fit_transform(df['location'])
# df['Country']=label.fit_transform(df['Country'])
# df['Work Type']=label.fit_transform(df['Work Type'])
# df['Job Title']=label.fit_transform(df['Job Title'])
# df['Role']=label.fit_transform(df['Role'])
# df['Company']=label.fit_transform(df['Company'])

In [137]:
df[['MinExp','MaxExp']] = df['Experience'].str.split(' to ', expand=True)
df[['MinSal','MaxSal']] = df['Salary Range'].str.split('-', expand=True)

In [138]:
df['MinSal'] = df['MinSal'].str.split('$', expand=True)[1]
df['MaxSal'] = df['MaxSal'].str.split('$', expand=True)[1]

In [139]:
df['MaxSal'] = df['MaxSal'].str.split('K', expand=True)[0].astype(int)
df['MinSal'] = df['MinSal'].str.split('K', expand=True)[0].astype(int)

In [140]:
df['MaxExp'] = df['MaxExp'].str.split(' ', expand=True)[0].astype('int64')
df['MinExp'] = df['MinExp'].str.split(' ', expand=True)[0].astype('int64')

In [141]:
df.drop(['Experience', 'Salary Range', 'Company Profile'], axis=1, inplace=True)

In [142]:
# Text preprocessing function
def preprocess_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    return " ".join(lemmatized_tokens)


In [143]:
def combine_text_cols(df, cols):
    return df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [144]:
df['combined_text'] = combine_text_cols(df, ['Job Description', 'Responsibilities', 'skills'])
df['Benefits_processed'] = df['Benefits'].apply(preprocess_text)
df['job_desc_processed'] = df['combined_text'].apply(preprocess_text)

In [145]:
tfidf_vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['job_desc_processed'] + " " + df['Benefits_processed'])
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index)

In [146]:
df = pd.concat([df, df_tfidf], axis=1)
df.drop(['Job Description', 'skills', 'Responsibilities', 'Benefits',
         'combined_text', 'job_desc_processed', 'Benefits_processed'], axis=1, inplace=True)

In [228]:
df['MinSal'].describe()

count    115552.000000
mean         60.013596
std           3.158085
min          55.000000
25%          57.000000
50%          60.000000
75%          63.000000
max          65.000000
Name: MinSal, dtype: float64

In [148]:
y = df[['MinSal', 'MaxSal']]
x = df.drop(['MinSal', 'MaxSal'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [149]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [215]:
y_pred = model.predict(X_test)

In [219]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [220]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 7.83706818968505
Mean Squared Error (MSE): 117.69247235202484
Root Mean Squared Error (RMSE): 10.848616149169665
R² Score: -0.025786782014057485


In [152]:
import pickle as pkl

In [153]:
# If you can retrain your model, add this before saving:
model.n_jobs = 1  # Reduces memory overhead
model.verbose = 0  # Disables logging

In [154]:
pkl.dump(model,open('model.pkl','wb'))

In [155]:
pkl.dump(tfidf_vectorizer,open('tfidf.pkl','wb'))

In [221]:
pkl.dump(encoders,open('encoder.pkl','wb'))

In [157]:
df

,Qualifications,location,Country,Work Type,Company Size,Job Title,Role,Company,MinExp,MaxExp,...,support,system,team,time,tool,transportation,tuition,user,wellness,work
0,9,127,87,1,42159,1,294,363,5,12,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,3,127,87,0,102180,21,43,775,0,14,...,0.000000,0.000000,0.0,0.179989,0.0,0.000000,0.000000,0.0,0.000000,0.143828
2,3,127,87,1,39814,17,366,592,4,13,...,0.000000,0.388876,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,2,127,87,0,48770,143,127,670,1,8,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.354910,0.000000
4,2,127,87,2,33699,69,120,143,5,12,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115935,1,105,58,1,18281,80,209,780,0,12,...,0.000000,0.380951,0.0,0.000000,0.0,0.171819,0.176176,0.0,0.000000,0.000000
115936,6,204,154,2,63621,56,165,296,2,14,...,0.168988,0.345235,0.0,0.165074,0.0,0.000000,0.000000,0.0,0.000000,0.131909
115937,4,11,198,3,114287,80,209,466,4,15,...,0.000000,0.372937,0.0,0.000000,0.0,0.000000,0.172470,0.0,0.126248,0.000000
115938,4,139,30,1,45009,51,350,519,5,15,...,0.000000,0.209056,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.141541,0.000000


In [158]:
df.columns

Index(['Qualifications', 'location', 'Country', 'Work Type', 'Company Size',
       'Job Title', 'Role', 'Company', 'MinExp', 'MaxExp',
       ...
       'support', 'system', 'team', 'time', 'tool', 'transportation',
       'tuition', 'user', 'wellness', 'work'],
      dtype='object', length=112)

In [ ]:
df

In [192]:
df2=pd.read_excel(r"C:\\Users\\vj753\\Downloads\\test_job.csv.xlsx")

In [193]:
df2.drop(index=0,axis=1,inplace=True)

In [194]:
cols_to_drop2 = ['Job Id', 'latitude', 'longitude', 'Job Posting Date',
                'Preference', 'Contact Person', 'Contact', 'Job Portal']
df2.drop(cols_to_drop2, axis=1, inplace=True)

In [195]:
df2 = df2.dropna()

In [222]:
df2.columns

Index(['Qualifications', 'location', 'Country', 'Work Type', 'Company Size',
       'Job Title', 'Role', 'Company', 'MinExp', 'MaxExp',
       ...
       'support', 'system', 'team', 'time', 'tool', 'transportation',
       'tuition', 'user', 'wellness', 'work'],
      dtype='object', length=112)

In [197]:


for col2 in categorical_cols:
    le = encoders[col2]
    # Apply transformation with a fallback for unseen categories
    df2[col2] = df2[col2].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1).astype(np.int16)    

In [198]:
df2[['MinExp','MaxExp']] = df2['Experience'].str.split(' to ', expand=True)
df2[['MinSal','MaxSal']] = df2['Salary Range'].str.split('-', expand=True)

In [199]:
df2['MinSal'] = df2['MinSal'].str.split('$', expand=True)[1]
df2['MaxSal'] = df2['MaxSal'].str.split('$', expand=True)[1]

In [200]:
df2['MaxSal'] = df2['MaxSal'].str.split('K', expand=True)[0].astype(int)
df2['MinSal'] = df2['MinSal'].str.split('K', expand=True)[0].astype(int)

In [201]:
df2['MaxExp'] = df2['MaxExp'].str.split(' ', expand=True)[0].astype('int64')
df2['MinExp'] = df2['MinExp'].str.split(' ', expand=True)[0].astype('int64')

In [202]:
df2.drop(['Experience', 'Salary Range', 'Company Profile'], axis=1, inplace=True)

In [203]:
df2['combined_text'] = combine_text_cols(df2, ['Job Description', 'Responsibilities', 'skills'])
df2['Benefits_processed'] = df2['Benefits'].apply(preprocess_text)
df2['job_desc_processed'] = df2['combined_text'].apply(preprocess_text)

In [204]:
tfidf_matrix2 = tfidf_vectorizer.transform(df2['job_desc_processed'] + " " + df2['Benefits_processed'])
df_tfidf2 = pd.DataFrame(tfidf_matrix2.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df2.index)

In [205]:
for col in df_tfidf.columns:
    if col not in df_tfidf2.columns:
        df_tfidf2[col] = 0

In [206]:
df_tfidf2 = df_tfidf2[df_tfidf.columns]

In [207]:
df2 = pd.concat([df2, df_tfidf2], axis=1)
df2.drop(['Job Description', 'skills', 'Responsibilities', 'Benefits',
         'combined_text', 'job_desc_processed', 'Benefits_processed'], axis=1, inplace=True)

In [208]:
y2 = df2[['MinSal', 'MaxSal']]
x2 = df2.drop(['MinSal', 'MaxSal'], axis=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.1, random_state=42)

In [209]:
X_test2 = X_test2[X_train.columns]

In [210]:
model.predict(X_test2)

array([[ 59.37, 106.21]])

In [211]:
X_test2

,Qualifications,location,Country,Work Type,Company Size,Job Title,Role,Company,MinExp,MaxExp,...,support,system,team,time,tool,transportation,tuition,user,wellness,work
2,9,102,114,4,84525,90,273,816,0,12,...,0,0.066898,0.077525,0,0,0.0,0,0.0,0,0.0


In [213]:
y_test2

,MinSal,MaxSal
2,61,104


In [ ]:
model.accuracy()